In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import pandas as pd

In [10]:
job_name = "Data Scientist"
country_name = "india"

job_url ="";
for item in job_name.split(" "):
    if item != job_name.split(" ")[-1]:
        job_url = job_url + item + "%20"
    else:
        job_url = job_url + item + "%20"

country_url ="";
for item in country_name.split(" "):
    if item != country_name.split(" ")[-1]:
        country_url = country_url + item + "%20"
    else:
        country_url = country_url + item

url = "https://www.linkedin.com/jobs/search?keywords={}&location={}&geoId=&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0".format(job_url,country_url)
url

'https://www.linkedin.com/jobs/search?keywords=Data%20Scientist%20Jobs&location=india&geoId=&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0'

In [11]:
# Creating a webdriver instance
driver = webdriver.Chrome("Driver/chromedriver.exe")
# Opening the url we have just defined in our browser
driver.get(url)

C:\Users\admin\AppData\Local\Temp\ipykernel_10204\185307237.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("Driver/chromedriver.exe")


This code is using the Selenium WebDriver to extract the number of jobs found in a job search result page.

The code first finds the number of jobs using a CSS selector, which selects the span element inside an h1 element on the page. It then gets the inner text of the span element using the get_attribute() method.

If the number of jobs contains a comma (,) in its string representation (i.e., if it has a value greater than or equal to 1,000), the code splits the string at the comma and multiplies the first element by 1000 to get the actual number of jobs. For example, if the inner text of the span element is "1,234 jobs found", the code will convert it to the integer value 1234.

If the number of jobs doesn't contain a comma, the code simply converts the inner text of the span element to an integer.

Finally, the code converts the resulting value to an integer using the int() function.

Note that this code assumes that the number of jobs is always represented as an integer with an optional comma separator. If this assumption is not true for a particular job search result page, the code may produce unexpected results or errors.
- Optional comma seperator - The code mentioned here assumes that the number of jobs returned by a job search result page is always represented as an integer with an optional comma separator.

For example, if the number of jobs returned is 1,234 then it is assumed to be represented as "1,234". If this assumption is not true for a particular job search result page, the code may produce unexpected results or errors.

For instance, if the number of jobs returned is represented as "1.2k" or "one thousand two hundred thirty-four," then the code may not be able to correctly parse or interpret this value as an integer, leading to unexpected results or errors.

Therefore, it's important to validate the format of the data being parsed and ensure that it conforms to the assumptions made in the code to prevent unexpected results or errors.


############################################################################################################################
This code is likely part of a web scraping script that is scraping job search results from a website. Here is an explanation of what the code is doing:

The code starts by initializing a variable i to the value 2.

It then enters a while loop that will continue to run as long as the value of i is less than or equal to half of the total number of jobs (jobs_num) plus one.

Within the while loop, the code uses the driver.execute_script function to execute a JavaScript command that scrolls the browser window to the bottom of the page.

The code then increments the value of i by 1, and prints out the current value of i and the percentage of the job search results that have been scrolled through so far.

The code then attempts to find and click on a "Load more results" button on the page, if one exists. If it is successful, the code waits for a short period of time (0.1 seconds) to allow the new results to load.

If the code is not able to find a "Load more results" button, it simply waits for a short period of time (0.1 seconds) and continues with the loop.

The purpose of this code is to scroll through the job search results and click on the "Load more results" button as needed in order to scrape all of the available job postings. The while loop continues until the script has scrolled through at least half of the total job search results (plus one), in order to ensure that all job postings are scraped.


In [12]:
#We find how many jobs are offered.
jobs_num = driver.find_element(By.CSS_SELECTOR,"h1>span").get_attribute("innerText")
if len(jobs_num.split(',')) > 1:
    jobs_num = int(jobs_num.split(',')[0])*1000
else:
    jobs_num = int(jobs_num)

jobs_num   = int(jobs_num)

#We create a while loop to browse all jobs. 
i = 2
while i <= int(jobs_num/2)+1:
    #We keep scrollind down to the end of the view.
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    i = i + 1
    print("Current at: ", i, "Percentage at: ", ((i+1)/(int(jobs_num/2)+1))*100, "%",end="\r")
    try:
        #We try to click on the load more results buttons in case it is already displayed.
        infinite_scroller_button = driver.find_element(By.XPATH, ".//button[@aria-label='Load more results']")
        infinite_scroller_button.click()
        time.sleep(0.1)
    except:
        #If there is no button, there will be an error, so we keep scrolling down.
        time.sleep(0.1)
        pass

In [13]:
#We get a list containing all jobs that we have found.
job_lists = driver.find_element(By.CLASS_NAME,"jobs-search__results-list")
jobs = job_lists.find_elements(By.TAG_NAME,"li") # return a list

#We declare void list to keep track of all obtaind data.
job_title_list = []
company_name_list = []
location_list = []
date_list = []
job_link_list = []

#We loof over every job and obtain all the wanted info.
for job in jobs:
    #job_title
    job_title = job.find_element(By.CSS_SELECTOR,"h3").get_attribute("innerText")
    job_title_list.append(job_title)
    
    #company_name
    company_name = job.find_element(By.CSS_SELECTOR,"h4").get_attribute("innerText")
    company_name_list.append(company_name)
    
    #location
    location = job.find_element(By.CSS_SELECTOR,"div>div>span").get_attribute("innerText")
    location_list.append(location)
    
    #date
    date = job.find_element(By.CSS_SELECTOR,"div>div>time").get_attribute("datetime")
    date_list.append(date)
    
    #job_link
    job_link = job.find_element(By.CSS_SELECTOR,"a").get_attribute("href")
    job_link_list.append(job_link)

In [14]:
job_data = pd.DataFrame({'Date': date_list,'Company': company_name_list,'Title': job_title_list,'Location': location_list,'Link': job_link_list})

In [15]:
job_data

,Date,Company,Title,Location,Link
0,2023-03-02,Falabella India,Machine Learning Engineer,Greater Bengaluru Area,https://in.linkedin.com/jobs/view/machine-lear...
1,2023-03-13,Supersourcing,AI/ML Engineer,"Bengaluru, Karnataka, India",https://in.linkedin.com/jobs/view/ai-ml-engine...
2,2023-03-14,Antal International,Data Scientist,"Bengaluru East, Karnataka, India",https://in.linkedin.com/jobs/view/data-scienti...
3,2023-03-06,Chegg Inc.,Senior Data Scientist,"New Delhi, Delhi, India",https://in.linkedin.com/jobs/view/senior-data-...
4,2023-03-20,Qualcomm,ML Engineer,"Bengaluru East, Karnataka, India",https://in.linkedin.com/jobs/view/ml-engineer-...
...,...,...,...,...,...
944,2023-03-05,Tata Consultancy Services,Micro Focus Developer,"Bengaluru, Karnataka, India",https://in.linkedin.com/jobs/view/micro-focus-...
945,2023-03-07,BatchService,Data Analyst,India,https://in.linkedin.com/jobs/view/data-analyst...
946,2023-03-19,Jobs for Humanity,"Lead Engineer (4+ years of experience in Java,...","Gurugram, Haryana, India",https://in.linkedin.com/jobs/view/lead-enginee...
947,2022-11-07,Elevate,Lead Developer,India,https://in.linkedin.com/jobs/view/lead-develop...


In [16]:
len(job_data)

949

In [20]:
job_data.to_csv('E:\DS - ExcelR\P3\job_details.csv') #csv

In [22]:
job_data.to_excel('E:\DS - ExcelR\P3\job_details.xlsx', index = False) #xlsx